# 概要
前面我们介绍了ANN，并且训练了第一个DNN，但是这只是一个非常简单的DNN，只有两个隐层。如果要处理非常复杂的问题，比如说在高分辨率的图片中检测上万个对象呢。就会训练一个更加深度的神经网络，比如说10层，每层包含有上千个神经元，上万个连接权值。这就会有很多问题：  
1.梯度消失问题（或者梯度爆发问题）导致底层网络很难训练。  
2.训练极慢。  
3.上亿的参数导致很容易过拟合。  
这里我们会逐个解决这些问题，解决问题1，然后对于问题2对比不同的优化器与一般gd，最后介绍一些常用的DNN的正则方法。  
# Vanishing/Exploding Gradients Problems
我们看到前面介绍的逆向算法，是对误差梯度从输出向输入逆向传播，每个GDstep都更新一次权值。  
不幸的是，梯度通常传播得越来越小，导致梯度更新几乎不在底层进行。这就叫梯度消失问题(Vanishing Gradients Problem）。同样的情况下，相反地高层的参数疯狂更新，导致算法偏离。这叫做爆发梯度问题(exploding gradients problem)。这在递归神经网络(recurrent NN)经常发生。更一般的说，这使得DNN不同层的学习速度大不相同。  
  
虽然这个问题被发现很久，但是知道2010年才有了重大进展，我们有了一些了解。一些sigma式的激活函数和随机初始化的技术才兴起，随机初始化使用（0,1）的正态分布。简单说，在这种情况下，每层网络的输出方差会越来越大。在前进过程中方差逐渐增加，直到顶层的激活函数饱和。这是由于逻辑函数的中值是0.5而不是0（hyperholic tangent 函数就是0中心所以在DNN表现比逻辑函数好很多）  
  
可以看到逻辑函数在输入很大或者很小的时候，导数几乎为0，所以当逆向传播到来时，几乎没有梯度存在，这就导致传播到底层时梯度几乎不剩。  
## Xavier 和 He 初始化
在那篇论文中提供了一个明显减缓这个问题的措施，在做预测的时候，和逆向的时候都需要都要能够保持信号。我们不希望信号消失，也不希望膨胀而饱和。我们要是每一层的输出信号的方差和前一层输出信号的方差一致，我们也需要梯度在逆向时方差也保持一致。实际上着是做不到的除非蹭的输入和输出数目相等，但是有个实践起来不错的措施，连接权值按照如下初始化：  
$Xavier初始化(当使用逻辑激活函数的时候)$  
$正太分布N(0,\sigma),其中\sigma=\sqrt{\frac{2}{n_{input}+n_{output}}}$  
$或者-r到r的均匀分布，r=\sqrt{\frac{6}{n_{input}+n_{output}}}$   
使用这种初始化策略也可以提高训练速度，这成就了DNN现在的成功，近年来一些论文还提供了类似的方法对应于不同的激活函数，比如hyperbolic tangent需要在值前面乘以4，ReLU需要乘以根号2.这些变种策略称为He初始化。  
默认的fully_connected()函数使用Xavier初始化
## 不饱和激活函数
后来实践证明ReLU表现比sigma好很多，就是因为它不饱和。  
不幸的是，ReLU并不理想，他有个称为dying ReLUs问题：当训练时，有些神经元实际上将会停止输出，输出一直是0，如果死了一样。有些情况下，你会发现网络中有一般的神经元都是死的，特别是在使用高学习率的时候。  
当训练的时候，如果一个神经元的权值被更新到使得输入总和为负的时候，他就开始输出0，一旦这种情况发生，他就不会再复活，因为他的梯度一直为0.  
为了解决这个问题，就要使用relu的变种，比如leaky ReLU.$LeakyReLU_{\alpha}(z)=max(\alpha z,z)$.

这里的$\alpha$表示leak(泄露)等级，典型设置为0.01，这个小斜率保证神经元不会死，他们会进入休眠期，然后偶尔会复活。  
实际上设置大一些的alpha比如0.2，结果可能还比0.01好，于是出现了随机泄露ReLU(randomized leaky ReLU)RReLU，alpha在训练时随机给出。这种办法在正则化的时候表现也很好。  
最后还有parametric leaky ReLU(参数化泄露ReLU)PReLU,这里alpha变成训练时学习的一个参数，就像误差逆向传播时更新别的参数一样。这个在大型图片数据中很有用，但是在小数据集很可能过拟合。  
还有最近的exponential linear unit(ELU指数线性单元).相比上面的变种，训练时间更短，并且在测试集表现更好。  
$ELU_{\alpha}(z)=\left{
\begin{array}{ll}
\alpha(exp(z)-1)&\text{if }z<0\\
z&\text{if }z\ge0\\
\end{array}
\right.$  
他很像ReLU,但是他在z小于0时为负值，允许输出的平均值接近0.有助于解决梯度消失问题。alpha定义了输入很小的时候输出是多少，通常设置为1.  
第二，他在z<0时梯度不为0，避免了神经元死亡。  
第三，他是可导函数。  
  
他的主要缺点是计算比较慢不如ReLU及其变体,主要是指数运算。但是作为偿还，他有很快的收敛速度，然而在实验时，ELU网络速度慢于ReLU。  
  
所以那个激活函数更加合适呢，一般来说ELU>leakyReLU>ReLU>tanh>logistic。然后要看运行表现，如果有空余时间可以CV这些激活函数。特别的在模型过拟合使用RReLU和数据集很大的时候使用PReLU

tf提供elu()函数。可以如下：
  hidden1 = fully_connected(X, n_hidden1, activation_fn=tf.nn.elu)  
tf没有提供leakyReLUs，但是这很容易自己定义：
def leaky_relu(z, name=None):  
    return tf.maxmum(0.01 * z, z, name=name)  
# batch规范化
尽管He和ELU的组合可以在训练开始时明显减少梯度消失和爆发问题，但是不能保证在训练中问题不会再次出现。  
2015年的一篇论文指出一个batchNormalization（batch规范化）的办法，指出这个问题和每一层的输入分布在训练中改变的有关（称为内部协方差偏移问题）。   
这个技术包括在激活函数之前的模型加入一个操作，简单地0中心化和规范化输入，然后缩放和偏移结果使用两个新的参数来控制，换句话说，这个操作就是让模型在理想的范围和中值下学习。   
为了实现变换，需要将输入的中值和标准差计算出来。也对每个mini-batch执行这样的操作：  
$\mu_B=\frac{1}{m_B}\sum_{i=1}^{m_B}\mathbf x^{(i)}$  
$\sigma_B^2=\frac{1}{m_B}\sum_{i=1}^{m_B}(\mathbf x^{(i)}-\mu _B)^2$  
$\hat{\mathbf x}^{(i)}=\frac{\mathbf x^{(i)}-\mu_B}{\sqrt{\sigma_B^2+\epsilon}}$  
$z^{(i)}=\gamma\hat{\mathbf x}^{(i)}+\beta$  
$\mu_B是mini-batch的中值$  
$\sigma_B是mini-batch的标准差$  
$m_B是mini-batch的样例数量$  
$\hat{\mathbf x}^{(i)}是中心化标准化的输入$  
$\gamma是层的缩放参数$  
$\beta是层的补偿偏移参数$  
$\epsilon是一个避免除零的极小数，通常是10^{-3}，称为平滑项$  
$\mathbf z^{(i)}是BN操作的输出，缩放化和偏移后的输出版本$

使用这种方法，梯度消失问题会大幅度下降，所以他们可以使用会饱和的激活函数，网络也对于权值初始化不那么敏感。还可以使用高的学习率，明显的提高训练速度，BN就像是一个正则化项，减少模型需要的其他正则化技术。  
BN也为模型加入了一些复杂度。产生了运行补偿，网络因为每层的额外计算，使得预测时速度变慢，所以如果希望预测很快，就要比较ElU+He和BN的实际表现。在第一次梯度下降的时候，寻找gamma和beta的值也比较慢，但是后面在找到不错的值后就比较快速了。
## 使用TF应用BN
TF提供了一个batch_normalization()函数，可以简单地变换输入项，但是需要自己计算中值和标准差，并且把他们作为参数，还需要操作缩放和偏移参数。这些可以使用batch_norm（）函数实现

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import batch_norm

n_inputs = 28 * 28
n_hidden1 = 300 
n_hidden2 = 100
n_outputs = 10 

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')

is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params = {
    'is_training': is_training,
    'decay': 0.99,
    'updates_collections': None
}

hidden1 = fully_connect(X, n_hidden1, scope='hidden1',
                normalizer_fn=batch_norm, normalizer_params=bn_params)

hidden2 = fully_connect(hidden1, n_hidden2, scope='hidden2',
                normalizer_fn=batch_norm, normalizer_params=bn_params)

logits = fully_connect(hidden2, n_outputs, activation_fn=None, scope='outputs',
                normalizer_fn=batch_norm, normalizer_params=bn_params)

is_training变量，用来告诉batch_norm()函数，是否要用来计算当前mini-batch的中值和标准差。  
然后是bn_params,用来定义batch_norm（）的参数，这个算法使用指数衰减exponential decay来计算滑动平均值，给出一个新的值$\upsilon$,滑动平均值$\hat{\upsilon}$通过$\hat{\upsilon}\gets\hat{\upsilon}\times decay+\upsilon\times(1-decay)$计算的。一个好的衰减值接近于1，比如0.9,0.99（数据集相对于mini-batch越大，9越多），最后update_collection用来设置batch_norm()函数是否需要在训练时的每次BN前升级滑动平均值。  
最后，我们创建全连接层函数，使用batch_norm()及其参数来在激活函数前规范化输入。  
注意在默认情况下，batch_norm（）会中心化，规范化，偏移化输入，单数不会缩放他们，这在层没有激活函数或者是ReLU时很重要，这样下个层的权值会更加关注缩放，但是对于其他激活函数，需要在bn_params中加入'scale':True。  
你还会注意到前面的代码在创建三个层时有大量重复，为了避免这种情况，可以创建一个argument scope如下：

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import batch_norm

n_inputs = 28 * 28
n_hidden1 = 300 
n_hidden2 = 100
n_outputs = 10 

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')

is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params = {
    'is_training': is_training,
    'decay': 0.99,
    'updates_collections': None
}

with tf.contrib.framework.arg_scope(
    [fully_connected],
    normalizer_fn=batch_norm,
    normalizer_params=bn_params):
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits = fully_connected(hidden2, n_outputs, scope='outputs', 
                            activation_fn=None)

对于少数隐层的模型来说，这种方法优化不了，但是对于10层以上，明显方便很多。  
剩下的就是构造损失函数，优化器，评估函数，建立保存点等等。  
执行层也一样，除了一点：需要确认是否使用BN。


In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        [...]
        for X_batch, y_batch in zip(X_batches, y_batches):
            sess.run(training_op,
                    feed_dict={istraing: True, X: X_batch, y: y_batch})
        accuracy_score = accuracy.eval(
            feed_dict={is_training: Fales...})
        print(accuracy_score)

# 梯度爬升
一个流行的解决梯度爆发问题的办法就是回溯的时候简单的爬升梯度，使得它不会小于一个阈值(这对于递归网络很有用).这就叫梯度爬升。一般来说现在倾向于使用BN。  
在tf中，优化器的最小化函数，关注梯度和应用他们，但是你可以替换优化器的梯度计算函数，然后增加一个爬升操作，使用clip_by_value()函数，最后创建一个操作来应用它使用apply_gradients()方法：

In [ ]:
threshold = 0.5
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
             for grad,var in grads_and_vars]
traing_op = optimizer.apply_gradients(capped_gvs)

只需要每步训练时都执行training_op就行。
# 重用已经训练好的层
从头开始训练一个非常大的DNN网络是不明智的，我们可以使用已经训练好的，任务相关的神经网络来放到底层，这叫做转移学习(transfer learning).可以加速我们的训练，而且需要更少的训练样例。这种方法在输入相似的情况下很有用。  
## 重用tf模型
如果原始模型已经使用TF训练好了，可以重载它，然后用来训练新的任务，

In [ ]:
[...] #construct the original model

with tf.Sess() as sess:
    saver.restore(sess, '/...ckpt')
    [...] #Train it on your new task

然而通常情况下只是想重用部分层一个简单的办法是重载一部分的变量数据：

In [ ]:
[...] #建立一个新模型，1-3隐层都使用原来定义相同的

init = tf.global_variables_initializer()

reuse_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                              scope='hidden[123]')

reuse_vars_dict = dict([(var.name, var.name) for var in reuse_vars])
original_saver = tf.Saver(reuse_vars_dict)  #用来重载原始模型

new_saver = tf.Saver() #用来保存新的模型

with tf.Session() as sess:
    sess.run(init)
    original_saver.restore('...ckpt')#重载1-3隐层的参数
    [...]   #训练新的模型
    new_saver.save('...new..ckpt')#保存新的模型

首先我们建立一个新的模型，确保前三个隐层和原始的模型隐层1-3一模一样。然后我们创建一个初始化节点。接着获得一个变量列表表示已经训练好的数据，并且制定需要获得的scope的变量。接着创建一个字典用来映射新模型的变量名和旧模型的变量名，通常是不变。然后创建Saver来保存新模型。  
对于很相似的任务，甚至可以只改变输出层，其他照搬。
## 从其他框架重用模型
如果模型使用其他框架训练，我们需要手动来加载权值，然后分配合适的值进去，这是一个冗长的工作

In [ ]:
original_w = [...] #加载从其他框架来的权值
original_b = [...] #加载从其他框架来的偏移

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
[...]  ##建立剩下的模型

#获取全连接网络的参数操作
with tf.variable_scope('', default_name='', reuse=True): #根域
    hidden1_weights = tf.get_variable('hidden1/wieights')
    hidden1_biase = tf.get_variable('hidden1/biases')
    
#创建节点用来分配权值和偏移值
original_weights = tf.placeholder(tf.float32, shape=(n_inputs, n_hidden1))
original_biases = tf.placeholder(tf.float32, shape=(n_hidden1))
assign_hidden_weights = tf.assign(hidden1_weights, original_weights)
assign_hidden_biases = tf.assign(hidden1_biase, original_biases)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    sess.run(assign_hidden_weights, feed_dict={original_weights: original_w})
    sess.run(assign_hidden_biases, feed_dict={original_biases: original_b})
    [...] #训练模型

## 冻结底层
一般来说，重用已经训练好的底层，需要在训练新模型的时候冻结他们。这样会让高层权值训练更容易。最简单的方法是，使用一个优化器来表示需要训练的变量包含在里面，排除底层变量：

In [ ]:
train_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                             scope='hidden[34]|outputs')
training_op = optimizer.minimize(loss, var_list=train_var)

## 缓存冻结层
一旦层被冻结，就不会改变，可以将样例在冻结层最顶方的输出缓冲。当训练的时候，这种办法使得训练速度大大提升，因为只需要对冻结层对全体样例算一遍输出，而不需要n_epoch（当内存足够时）：

In [ ]:
hidden2_outputs = sess.run(hidden2, feed_dict={X: X_train})

当训练的时候，就不需要对训练集建立batches，转而对隐层2的输出做batch

In [ ]:
import numpy as np

n_epochs = 100
n_batches = 500

for epoch in range(n_epochs):
    shuffled_idx = rnd.permutation(len(hidden2_outputs))
    hidden2_batches = np.array_split(hidden2_outputs[shuffled_idx], n_batches)
    y_batches = np.array_split(y_train[shuffled_idx], n_batches)
    for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
        sess.run(training_op, feed_dict={hidden2: hidden2_batch, y: y_batch})

## 调整，忽略和替换顶层
原始模型的输层通常都会被替换掉，因为他们对新任务多数情况下没什么帮助，通常输出的数组长度和新任务不匹配。  
相似的，很多顶层也不像底层那么适用，高层的特征不适合新的任务，所以需要选择合适的重用层数目。  
首先，需要冻结所有重用的层，然后训练你的模型并观察表现，然后从上向下解冻1到2层，使得逆向算法可以调整他们，然后观察表现是否提升，训练数据越多，需要解冻的层数就越多。  
如果不能得到好的表现，就需要使用少量数据，尝试忽略顶层(s)然后冻结所有剩下的层，再测试。反复迭代直到找到合适的数量。如果有足够的数据，就尝试替换顶层而不是忽略他们，甚至增加一些隐层。
## 模型动物园
在哪里可以找到相似任务的网络模型呢，首先需要在自己的目录中找，所以需要经常保存自己的模型，并组织起来。  
另一个可选项是搜索模型动物园model zoo，很多人将他们的模型在这里共享。  
TF有自己的可用model zoo。有很多有名图像分类网络VGG,Inception,ResNet,以及他们的代码，已经训练好的模型，下载公用的图像数据集的工具。  
另一个是Caffe's Model Zoo.他也包含了很多机器视觉模型。还有专门的转换器可以在github上搜索到caffe-tensorflow。
## 非监督式预训练
假设现在你要处理一个复杂任务，没有太多的标签的数据，那可能就无法找到相似任务的模型。  
首先，你当然是需要获取更多带有标签的数据，但是这可能很难而且很昂贵，那么可能就要使用非监督式预训练方法(unsurpervised pretraining)。如果你有足够的无标签数据，可以从低层一层层地训练他们，使用非监督式特征检测算法，比如约束boltzmann机或者其他算法，每一层都在前面冻结的情况下训练，一旦左右层训练完成，你就能使用监督学习(比如逆向)算法来调整网络。  
这是一项冗长的工作，但是这通常运行良好。实际上，这个技术是2006年研发出来并且在DL中取得巨大成功，直到2010年，非监督式训练变成了深度网络的家常便饭，在梯度消失问题解决前，几乎是所有DNN的训练手段，然而，非监督式预训练还是一个用来处理复杂问题的好办法，在没有相似模型可以重用，没有标签数据而拥有足够的非标签数据时。
## 在辅助任务上预训练
最后一个训练的可选项是在辅助任务上能够容易的获取标签数据，然后重用他们的底层来用在实际的任务中。  
比如说想要建立一个系统来识别人脸，你大概有一些单独的照片，但是不足以训练一个好的分类器。获得上万人的图片是不实际的，然而，你可以获得一些随机的人物图片，通过网络，然后训练第一个神经网络来识别不同的照片是不是同一个人。这样的网络可以学习分辨脸部的好特征，所以重用底层使得你可以在训练数据不多的情况下训练一个好的分类器。  
同样的获取无标签数据是简单的，而标记他们是困难的。在这种情况下，通常的手段是对所有标记都设置为'good'，然后通过修改他们生成一些和他们差别的数据标记为'bad'。然后训练成为区分好和坏的第一个模型。    
另一个方法是对样例训练得到分数，然后使用一个损失函数来确认哪些是好的，哪些是坏的，这叫做最大分隔学习，max margin learning。  
# 更快的优化器
训练大型的DNN速度很慢，目前我们找到四个方法来加快：初始化策略，激活函数策略，BN，重用模型。还有一个办法就是使用比GD更加快速的优化器。

## Momentum optimization势力优化
想象保龄球在一个光滑表面的缓坡：一开始他速度很慢，然后势力越来越强直到到达垂直方向的最快速度(考虑空气阻力).这是势力优化的一个基本思路。通常的GD使用一个定步长，最后需要长时间才能到达底部。  
调用GD简单地更新权值$\theta$，直接用它减去损失函数当前的对所有权值的梯度和学习率的乘积，也就是$\theta\gets\theta-\eta\bigtriangledown_{\theta}J(\theta)$。这里没有关注原来的梯度值，如果当前梯度小，那么更新就变慢。  
势力优化关注了上一次的梯度值：对于每一次迭代，向当前的梯度加入一个势力向量$\mathbf m$，然后更新时减去当前的$\mathbf m$，换句话说，梯度变成了加速度，而不是一个速度，为了适应一些问题，且避免势力变得非常大，算法提供了一个参数$\beta$称为momentum，通常设置0-1，典型值0.9.  
$\mathbf m\gets\beta\mathbf m+\eta\bigtriangledown_{\theta}J(\theta)$  
$\theta\gets\theta-\mathbf m$  
可以简单的计算出，当梯度恒定的时候，他的最终速度(比如说更新权值的速度)等于$梯度\times\eta\times\frac{1}{1-\beta}$  
  
在不使用BN的dnn中，上层通常截止在输入范围非常不同的情况，所以这种算法很有帮助，还能逃离局部最优。因为这个势力，优化器可能会过头，然后返回再过头，多次后才停下，这是系统有阻尼的一个优点。  
  
在TF中可以无脑地应用MO，MomentumOptimizer.  
  
它的缺点是，增加了一个需要调节的参数，但是0.9通常运行良好。
## Nesterov Accelerated Gradient
MO的一个变种比原来的更加快速，NAG，它测量损失函数的梯度不是使用当前位置，而是朝着momentum的方向：  
$\mathbf m\gets\beta\mathbf m+\eta\bigtriangledown_{\theta}J(\theta+\beta\mathbf m)$  
$\theta = theta-\mathbf m$  
这一点微调是因为势力方向通常指向正确方向，所以他能加速，还有助于减小震荡。  
NAG的调用是在MO上加入use_nesterov=True实现的。
## AdaGrad
再考虑碗型问题，GD在初始位置快速下降，然后在底部渐渐停下，如果一个算法可以提早正确的检测出全局最优所在的方向肯定最好。AdaGrad算法使用陡峭的情况来缩放梯度矩阵。  
$\mathbf s\gets\mathbf s+\bigtriangledown_{\theta}J(\theta)\otimes\bigtriangledown_{\theta}J(\theta)$  
$\theta\gets\theta-\eta\bigtriangledown_{\theta}J(\theta)\oslash\sqrt{\mathbf s+\epsilon}$  
第一步是向向量$\mathbf v$加入梯度的平方，对于每个s:$s_i\gets s_i+(\partial/\partial\theta_iJ(\theta))^2$，换句话说每个s都加上了对theta的偏导的平方，如果损失函数在第i个方向很陡，那么他就会越来越大。  
第二步几乎就是理想的GD，但是被缩放了一个$\sqrt{\mathbf s+\epsilon}(\oslash表示元素除，\epsilon是平滑项，避免零除，通常设置10^{-10}$  
总的来说，算法按照学习率衰减，但是在梯度陡峭的地方下降很快，这叫做适应学习率adaptive learning rate。有助于直接向全局最优点更新参数，一个额外的好处就是它不怎么需要调整学习率$\eta$   
AG通常在二次问题上表现很好，但是不好的事他在神经网络里进场会过早的停止。学习率衰减太快，导致在达到全局最优前就停止学习，所以即使TF提供了这个优化器，也不能使用它来训练DNN。

## RMSProp
即使AG慢的太快，甚至不能覆盖最优点，但是RMSProp修正了他的加项    
$\mathbf s\gets\beta\mathbf s+(1-\beta)\bigtriangledown_{\theta}J(\theta)\otimes\bigtriangledown_{\theta}J(\theta)$  
$\theta\gets\theta-\eta\bigtriangledown_{\theta}J(\theta)\oslash\sqrt{\mathbf s+\epsilon}$    
这里的衰减率beta通常设置为0.9，没错这里又引进了一个需要调整的参数，不过还好0.9通常表现不错。  
除非是非常简单的问题，他的表现通常比AG甚至MO还有NAG都要好。事实上，在这种优化器在Adam optiization出现之前一直是主流。  
## Adam Optimization 
Adam，是基于adaptmive moment estimation，结合了MO和RMSprop的思路，他保持过去梯度的指数衰减，并且能像RMSPROP土塘衰减过去梯度的平方平均值。  
$\mathbf m\gets\beta_1\mathbf m+(1-\beta_1)\bigtriangledown_{\theta}J(\theta)$  
$\mathbf s\gets\beta_2\mathbf s+(1-\beta_2)\bigtriangledown_{\theta}J(\theta)\otimes\bigtriangledown_{\theta}J(\theta)$  
$\mathbf m\gets\frac{\mathbf m}{1-\beta_1^T}$  
$\mathbf s\gets\frac{\mathbf s}{1-\beta_1^T}$  
$\mathbf \theta\gets\theta-\eta\mathbf m\oslash\sqrt{\mathbf s+\epsilon}$

实际上adam是一个适应学习率算法，只需要调节eta，通常取0.001，比GD快速很多。  
  
    
所有上述的优化器都是用来训练稠密的参数矩阵。如果要使模型在运行时快速，那就要使用稀疏矩阵。  
一个办法是许梿模型，使权值很小。  
另一个办法是应用强l1正则化，使尽量多的权值为0.  
但是这些办法都不效率，最后一个办法是使用对偶平均，或者称为Follow the regularized leader（FTRL），当使用L1正则时，这个技术就会产生非常稀疏的模型。TF有FTRLOptimizer类。
## 学习率计划
找到一个好的学习率是非常有技巧的。  
通常需要：  
分段地使用常数学习率，在表现不错的情况下使用他们：  
观察计划表现，测算一些验证误差（就像提前结束那样），当提前结束前使用因子来减小学习率。  
指数型学习计划：这通常表现不错，

# 通过正则化避免过拟合
## 提前结束
在验证集上的效果不提升时结束训练。  
一个办法是使用TF来评估验证集表现，然后保存最优点。  
虽然提前结束实践起来很不错， 但是你可以使用其他技术来使得网络更好。 
## L1和L2正则化
一个办法是在TF中的损失函数中加入项，但是TF有更加方便的办法，创建一个用来计算L1正则损失的节点，来关联到各个权值。然后计算的时候TF自动加入这些节点。


In [ ]:
with arg_scope(
    [fully_connected],
    weights_regularizer=tf.contrib.layers.l1_regularizer(scale=0.01))
    [...]
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([base_loss] + reg_losses, name='loss')

## Dropout
即使是艺术即的神经网络也能在加入dropout的时候提高正确率，  
在每个训练步骤里，每个神经元都有概率p被暂时的停止：将在这个训练步骤中被忽略，但是可能在下一步被激活，p被称为忽略率，通常设置为0.5.训练完成后就不再被忽略。  
这个很让人惊讶，神经元在训练时被忽略不会导致他的角色被邻近的神经元适应。
## 最大标准正则化Max-Norm Regularization
对于每一个神经炎，对权值进行约束，$||\mathbf w||_2<r$，其中r是最大标准，$||\cdot||_2$是l2项。
我们通常的应用时计算$||\mathbf w||_2$在每步训练后并且更新他，如果必要的话$\mathbf w\gets\mathbf w\frac{r}{||\mathbf w||_2}$.
减少r相当于增强正则化，有助于减少过拟合。这个方法还有助于梯度消失和膨胀问题。  
tf有专门的max_norm_regularizer函数。注意MN方法不需要在loss函数中添加项。  
## 数据扩大
最后一个正则化办法，是数据扩大，包括从已经存在的样例中生成新源里，人为的扩大数据集，这样会减少过拟合，这是制造逼真的训练样例。通过简单的加入白噪声。比如对于一个图片分类模型，可以偏移，旋转，修改尺寸，等等来制造新样例。tf就提供了这些图像处理的API。